## Access DB2 on Cloud using Python

This notebook illustrates how to access your database instance using Python by following the steps below:
1. Import the `ibm_db` Python library
1. Identify and enter the database connection credentials
1. Create the database connection
1. Create a table
1. Insert data into the table
1. Query data from the table
1. Retrieve the result set into a pandas dataframe
1. Close the database connection

## 1: Import the `ibm_db` Python library

In [2]:
import ibm_db

## 2: Identify the database connection credentials

Connecting to dashDB or DB2 database requires the following information:
* Driver Name
* Database name 
* Host DNS name or IP address 
* Host port
* Connection protocol
* User ID
* User Password

In [3]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "kjb18772"        # e.g. "abc12345"
dsn_pwd = "bz0slrxg@vckz0qh"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

## 3: Create the database connection

In [23]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  kjb18772 on host:  dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net


## 4: Create a table in the database

In this step we will create a table in the database with following details:

<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">


In [33]:
# Let's firstly drop the Instructor table in case it exists from the previous attempt
dropquery = "drop table INSTRUCTOR"
dropStmt =ibm_db.exec_immediate(conn, dropquery)

In [34]:
# Construct the Create Table DDL statement - replace the ... with rest of the statement
createQuery = "CREATE TABLE INSTRUCTOR(id integer PRIMARY KEY NOT NULL, fname VARCHAR(20), lname VARCHAR(20), city VARCHAR(20), ccode CHARACTER(2))"

# Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

## 5: Insert data into the table

In this step we will insert some rows of data into the table. 

The INSTRUCTOR table we created in the previous step contains 3 rows of data:

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">

We will start by inserting just the first row of data, i.e. for instructor Rav Ahuja 


In [35]:
# Construct the query - replace ... with the insert statement
insertQuery = "INSERT INTO INSTRUCTOR(id, fname, lname, city, ccode) VALUES ('1', 'Rav', 'Ahuja', 'TORONTO', 'CA'), ('2', 'Raul', 'Chong', 'Markham', 'CA'), ('3', 'Hima', 'Vasudevan', 'Chicago', 'Us')"

# execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

## 6: Query data in the table
In this step we will retrieve data we inserted into the INSTRUCTOR table.

In [44]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only) - replace ... with your code
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [45]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
    print(" ID:", ibm_db.result(selectStmt, 0),  "FNAME:", ibm_db.result(selectStmt, "FNMAE"))

 ID: 2 FNAME: None
 ID: 3 FNAME: None


Bonus: now write and execute an update statement that changes the Rav's CITY to MOOSETOWN

In [47]:
UpdateQuery = "update instructor set city = 'moosetown' where fname = 'Rav'"
UpdateStmt = ibm_db.exec_immediate(conn, UpdateQuery)

## 7: Retrieve data into Pandas
In this step we will retrieve the contents of the INSTRUCTOR table into a Pandas dataframe

In [48]:
import pandas
import ibm_db_dbi

In [49]:
# Connection for Pandas
pconn = ibm_db_dbi.Connection(conn)

In [51]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [52]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,moosetown,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,Us


In [53]:
pdf.shape

(3, 5)

## 8: Close the Connection
We free all resources by closing the connection. Remember that it is always important to close connections so that we can avoid unused connections taking up resources.

In [55]:
ibm_db.close(conn)

True